In [38]:
import wget
import os
from datetime import datetime as dt
import pandas as pd
import re
from sqlalchemy import create_engine
import pymysql
import yaml
from os import path
import os
with open('../config.yml') as f:
    config = yaml.load(f)

project_dir = config['PROJECT']
project_db_dir = config['DATABASE']

#Download and Set the name of the file as the date
file_link = "ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz"
current_date = dt.now().strftime("%d_%m_%Y")
file_name = current_date + "_ClinVar_RawFile.txt.gz"
wget.download(file_link, file_name)
!gunzip {file_name} -f

#Read the file
#data = pd.read_csv("ClinVar_RawFile.txt.gz", compression='gzip', sep="\t")
df = pd.read_csv(file_name.split(".gz")[0], sep="\t")

#ClinVar Data where GeneID=-1 in GRCh37 assembly
#df_no_geneID = df.loc[(df["Assembly"] == "GRCh37") & (df["GeneID"] == -1)]
#df_no_geneID.to_csv("ClinVar_NoGeneID_GRCh37.csv", index=False, sep="\t")

!rm {file_name.split(".gz")[0]}

/home/kaplanlab/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
#ClinVar Filters step by step
column_list = ["GeneID", "#AlleleID", "GeneSymbol", "RS# (dbSNP)", "RCVaccession", 
               "VariationID", "Type", "Name", "ClinicalSignificance", "LastEvaluated", 
               "Cytogenetic", "ReviewStatus", "PhenotypeList"]

#ClinVar A.acid Subs. Data
df = df.loc[(df["Assembly"] == "GRCh37") & (df["GeneID"] != -1)]
dfAaSubs = df[df["Name"].str.contains(" \(p.", regex=True, na=False)]
dfNM_Yes = dfAaSubs[dfAaSubs["Name"].str.contains("^NM_", regex=True, na=False)]
df_selected = dfNM_Yes.loc[:, column_list]
df_selected["nm_id"] = df_selected["Name"].str.split('\(', expand=True)[0].str.split('.', expand=True)[0]
df_selected["variation"] = df_selected["Name"].str.split(' ',expand=True)[1]
df_selected["position"] = df_selected["variation"].str.extract('([0-9][0-9]*)', expand=True)


In [40]:
#Conversion from NM ID to NP ID
conversion_table = pd.read_csv(path.join(project_db_dir, 'mapping', 'NM_NP_GeneID.list'), sep="\t", header=None)
conversion_table.columns = ["nm_id", "np_id", "gene_id"]
conversion_table.columns

#Merging two dataframes
clinvar = pd.merge(df_selected, conversion_table, on='nm_id', how="inner")
clinvar["np_id"] = clinvar["np_id"].str.split(".", expand=True)[0]
clinvar = clinvar.drop("gene_id", axis=1)
clinvar.columns = ["gene_id", "allele_id", "symbol", "rs_number", "rcv_accession", 
                   "variation_id", "variant_type", "name", "clinical_significance", 
                   "last_updated", "cytogenetic", "review_status",
                   "phenotypes", "nm_id", "variation", "position", "np_id"]
clinvar["rs_number"] =  "rs" + clinvar["rs_number"].astype(str)

In [50]:
#MySQL DB Connection
con = pymysql.connect(host=config['MYSQL_HOST'], user=config['MYSQL_USER'], passwd=config['MYSQL_PASSWD'], 
                      db=config['MYSQL_DB'])

cur = con.cursor()

# MAKE THE ClinVar table EMPTY 
cur.execute('TRUNCATE TABLE clinvar')
con.commit()

# MySQL Migration
engine = create_engine("mysql+pymysql://{user}:{pw}@{server}/{db}"
                       .format(server=config['MYSQL_HOST'], user=config['MYSQL_USER'], pw=config['MYSQL_PASSWD'], 
                               db=config['MYSQL_DB']))

clinvar.to_sql('clinvar', engine, if_exists='replace', index=True, index_label='clinvar_id')
